<a href="https://colab.research.google.com/github/MaroonBlue/Speech_Commands_with_RNNs/blob/main/notebooks/main_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

## Cloning repository

In [ ]:
%cd /content/
!rm -r ./Speech_Commands_with_RNNs/
!git clone https://github.com/MaroonBlue/Speech_Commands_with_RNNs.git
%cd Speech_Commands_with_RNNs/

/content
rm: cannot remove './Speech_Commands_with_RNNs/': No such file or directory
Cloning into 'Speech_Commands_with_RNNs'...
remote: Enumerating objects: 293, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 293 (delta 44), reused 58 (delta 29), pack-reused 213
Receiving objects: 100% (293/293), 1.39 MiB | 9.15 MiB/s, done.
Resolving deltas: 100% (165/165), done.
/content/Speech_Commands_with_RNNs


## Installing packages

In [ ]:
!pip install -r requirements.txt
!pip install -U -q tensorflow tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29623 sha256=c890ea73d47c8eafde308926faa6b4979580a1bfb783a17320137da38e1f727b
  Stored in direc

# Downloading dataset

## Download form URL

In [ ]:
!python utils/download_dataset.py

2023-04-24 18:16:19.973780: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 18:16:21.428617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1121103842/1121103842 [==============================] - 24s 0us/step
Moved 6798 files to the validation directory
Moved 6835 files to the test directory


## Generate and save training and validation data

In [ ]:
!python ./utils/dataset.py

2023-04-24 18:19:06.305697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 18:19:08.494141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Training dataset: 100% 22536/22536 [06:35<00:00, 56.99it/s]
Validation dataset: 100% 6798/6798 [01:36<00:00, 70.46it/s]


## Copy .npy files to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!cp -R saved_data ../drive/MyDrive/project_data

Mounted at /content/drive/


## Copy from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/project_data')
!cp -R ../content/drive/project_data saved_data

ValueError: ignored

## Copy from local

In [ ]:
!cp -R data ../data

In [ ]:
!cp -R ../data data

In [ ]:
!cp ./saved_data/X_t.npy ../X_t.npy
!cp ./saved_data/y_t.npy ../y_t.npy
!cp ./saved_data/X_v.npy ../X_v.npy
!cp ./saved_data/y_v.npy ../y_v.npy

In [ ]:
!cp ../X_t.npy ./saved_data/X_t.npy
!cp ../y_t.npy ./saved_data/y_t.npy
!cp ../X_v.npy ./saved_data/X_v.npy
!cp ../y_v.npy ./saved_data/y_v.npy

# Experiments

## Setup and imports

In [ ]:
%cd Speech_Commands_with_RNNs/

/content/Speech_Commands_with_RNNs


In [ ]:
import os
import wandb
import tensorflow as tf
import sys
import numpy as np

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from numpy import load

sys.path.append("./")
from utils.utils import set_seeds, make_configs, step_decay
from models.test_model import get_test_model
from models.CNN_LSTM import CNN_LSTM1
from models.GRU import GRU1, GRU2, GRU3
from models.LSTM import LSTM1, LSTM2, LSTM3
from models.simpleRNN import simpleRNN1, simpleRNN2, simpleRNN3

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

## Configs and data

In [ ]:
ENTITY = 'wo-team'
PROJECT = 'dl-rnn-audio'
GROUP = 'simple-lstm-gru'
NAME = '_'
SAVE_PATH = 'weights/'

models = {
    'CNN_LSTM': CNN_LSTM1,
}

base_config = {
    'dataloader': {
        'sample_rate': 8000,
        'unknown_silence_samples': 2000,
        'seed': 0,
        'batch_size': 128,
        'convert_to_image': False,
    },
    'training': {
        'n_epochs': 50,
        'dropout': 0.3,
    },
    'compile':{
        'loss': 'sparse_categorical_crossentropy',
        'optimizer': 'adam',
        'metrics': ['accuracy', 'sparse_categorical_accuracy']
    },
    'model': {
        'architecture': 'CNN_LSTM',
        'model_init': None,
        'id': None,
        'save_path': None,
    },
    'early_stopper':{
        'monitor': 'val_sparse_categorical_accuracy',
        'min_delta': 0.001,
        'patience': 4,
        'verbose': 1,
        'start_from_epoch': 10,
        'restore_best_weights': True,
    },
    'checkpointer':{
        'monitor': 'val_sparse_categorical_accuracy',
        "verbose": 1,
        'save_best_only': True
    },
    'scheduler': LearningRateScheduler(step_decay),
    'other':{
            'num_classes':12,
    }
}

combinations = {
    'seeds': {
        'dict_path': ['dataloader', 'seed'],
        'values': [0, 1, 2]
    },
    'dropout': {
        'dict_path': ['training', 'dropout'],
        'values': [0.3, 0.5]
    }

}



configs = make_configs(base_config, combinations)

LOAD_PTH = "./saved_data/"
X_t = load(LOAD_PTH + "X_t.npy")
y_t = load(LOAD_PTH + "y_t.npy")
X_v = load(LOAD_PTH + "X_v.npy")
y_v = load(LOAD_PTH + "y_v.npy")
y_t = np.argmax(y_t, axis=1).transpose()
y_v = np.argmax(y_v, axis=1).transpose()

## Training

In [ ]:
start_config = int(input("Provide ID of first config: "))

for i, config in enumerate(configs):
    i = i + 54
    if i < start_config:
        continue

    set_seeds(config['dataloader']['seed'])
    config['model']['id'] = i
    NAME = str(config['model']['id']) + config['model']['architecture'] + "_seed" + str(config['dataloader']['seed']) +"_dropout"+str(config['training']['dropout']).replace(".", ",")
    config['model']['model_init'] = models[config['model']['architecture']]

    wandb.init(
        project = PROJECT,
        entity = ENTITY,
        group = GROUP,
        name = NAME,
        config = config)
    
    l = len(configs)
    print(f"---------------\nConfig {i+1}/{l}\n---------------\n\n")
    print('Running config:', config, "\n")

    input_shape = X_t.shape[1:]

    model = config['model']['model_init'](input_shape=input_shape, output_nodes=12, dropout=config['training']['dropout'])

    model.compile(**config["compile"])
    earlystopper = EarlyStopping(**config["early_stopper"])
    checkpointer = ModelCheckpoint(NAME+'.h5', **config["checkpointer"])
    lrate = config["scheduler"]

    history = model.fit(
                X_t,
                y_t,
                epochs=config['training']['n_epochs'],
                validation_data=(X_v, y_v),
                batch_size=config['dataloader']['batch_size'],
                shuffle=True,
                callbacks=[
                    earlystopper, 
                    checkpointer, 
                    lrate,
                    WandbMetricsLogger(log_freq=5),
                    WandbModelCheckpoint("weights/wandb")
                ])
    save_path = os.path.join(SAVE_PATH, NAME)
    model.save(save_path)

    wandb.finish()

wandb: Currently logged in as: martyna-majchrzak19 (wo-team). Use `wandb login --relogin` to force relogin


---------------
Config 55/6
---------------


Running config: {'dataloader': {'sample_rate': 8000, 'unknown_silence_samples': 2000, 'seed': 0, 'batch_size': 128, 'convert_to_image': False}, 'training': {'n_epochs': 50, 'dropout': 0.3}, 'compile': {'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam', 'metrics': ['accuracy', 'sparse_categorical_accuracy']}, 'model': {'architecture': 'CNN_LSTM', 'model_init': <function CNN_LSTM1 at 0x7fe9110074c0>, 'id': 54, 'save_path': None}, 'early_stopper': {'monitor': 'val_sparse_categorical_accuracy', 'min_delta': 0.001, 'patience': 4, 'verbose': 1, 'start_from_epoch': 10, 'restore_best_weights': True}, 'checkpointer': {'monitor': 'val_sparse_categorical_accuracy', 'verbose': 1, 'save_best_only': True}, 'scheduler': <keras.callbacks.LearningRateScheduler object at 0x7fe9940cfd00>, 'other': {'num_classes': 12}} 

Changing learning rate to 0.001
Epoch 1/50
160/160 [==============================] - ETA: 0s - loss: 1.3057 - accuracy: 0.5559 

wandb: Adding directory to artifact (./weights/wandb)... Done. 0.0s


160/160 [==============================] - 804s 5s/step - loss: 1.3057 - accuracy: 0.5559 - sparse_categorical_accuracy: 0.5559 - val_loss: 1.3875 - val_accuracy: 0.5048 - val_sparse_categorical_accuracy: 0.5048 - lr: 0.0010
Changing learning rate to 0.001
Epoch 2/50
160/160 [==============================] - ETA: 0s - loss: 0.4823 - accuracy: 0.8469 - sparse_categorical_accuracy: 0.8469
Epoch 2: val_sparse_categorical_accuracy improved from 0.50481 to 0.70197, saving model to 54CNN_LSTM_seed0_dropout0,3.h5


wandb: Adding directory to artifact (./weights/wandb)... Done. 0.0s


160/160 [==============================] - 746s 5s/step - loss: 0.4823 - accuracy: 0.8469 - sparse_categorical_accuracy: 0.8469 - val_loss: 0.8564 - val_accuracy: 0.7020 - val_sparse_categorical_accuracy: 0.7020 - lr: 0.0010
Changing learning rate to 0.001
Epoch 3/50
 80/160 [==============>...............] - ETA: 5:24 - loss: 0.3686 - accuracy: 0.8824 - sparse_categorical_accuracy: 0.8824